In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pathlib import Path
import glob

# Import helper functions
import sys
sys.path.append('../src')
from utils import add_landcover_labels, summarize_landcover

# Set style cho matplotlib
plt.style.use('seaborn')
sns.set_palette("husl")

# Đọc tất cả file CSV từ thư mục raw_exports
data_path = Path('../data/raw_exports')
all_files = glob.glob(str(data_path / "*.csv"))

# Đọc và gộp tất cả file
dfs = []
for file in all_files:
    df = pd.read_csv(file)
    dfs.append(df)
    
df_combined = pd.concat(dfs, ignore_index=True)

print(f"Tổng số mẫu: {len(df_combined):,}")
print(f"Số cột: {len(df_combined.columns)}")
print("\nCác cột trong dataset:")
print(df_combined.columns.tolist())

# Phân tích phân bố Land Cover và mối quan hệ với ngập lụt

Notebook này phân tích phân bố các loại lớp phủ mặt đất (land cover) từ dữ liệu ESA WorldCover và mối quan hệ của chúng với ngập lụt ở Việt Nam.

## Nội dung:
1. Import thư viện và load dữ liệu
2. Phân tích phân bố lớp phủ
3. Tỷ lệ ngập theo loại lớp phủ
4. Bản đồ tương tác

## Phân tích phân bố lớp phủ

Phân tích phân bố các loại lớp phủ trong dataset và tỷ lệ ngập của từng loại:

In [ ]:
# Thêm tên lớp phủ và phân tích
df_combined = add_landcover_labels(df_combined)
summary = summarize_landcover(df_combined)

# Vẽ biểu đồ phân bố
plt.figure(figsize=(15, 6))
sns.barplot(data=summary.reset_index(), 
           x='land_cover_name', 
           y='total_samples',
           alpha=0.6)
plt.xticks(rotation=45, ha='right')
plt.title('Phân bố các loại lớp phủ trong dataset')
plt.ylabel('Số lượng mẫu')
plt.xlabel('Loại lớp phủ')
plt.tight_layout()
plt.show()

# Vẽ biểu đồ tỷ lệ ngập
plt.figure(figsize=(15, 6))
sns.barplot(data=summary.reset_index(), 
           x='land_cover_name', 
           y='flood_ratio',
           alpha=0.6)
plt.xticks(rotation=45, ha='right')
plt.title('Tỷ lệ ngập (%) theo loại lớp phủ')
plt.ylabel('Tỷ lệ ngập (%)')
plt.xlabel('Loại lớp phủ')
plt.tight_layout()
plt.show()

# In bảng tổng kết
print("\nBảng tổng kết chi tiết:")
print(summary.round(2).to_string())

## Bản đồ tương tác

Tạo bản đồ tương tác để xem phân bố không gian của các mẫu và lớp phủ:

In [ ]:
# Tạo bản đồ centered ở Việt Nam
m = folium.Map(location=[16.0, 106.0], zoom_start=5)

# Tạo color map cho các loại lớp phủ
unique_landcover = df_combined['land_cover'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(unique_landcover)))
color_map = dict(zip(unique_landcover, [f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}' 
                                      for r, g, b, _ in colors]))

# Lấy mẫu ngẫu nhiên 1000 điểm để vẽ (để tránh quá tải)
sample_size = 1000
df_sample = df_combined.sample(sample_size, random_state=42)

# Thêm markers cho từng điểm
for idx, row in df_sample.iterrows():
    color = color_map[row['land_cover']]
    popup_text = f"""
    Lớp phủ: {row['land_cover_name']}<br>
    Ngập: {'Có' if row['flood']==1 else 'Không'}<br>
    Độ cao: {row['elevation']:.1f}m<br>
    Độ dốc: {row['slope']:.1f}°
    """
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        popup=popup_text,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m)

# Thêm legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px; border: 2px solid grey;">
<h4>Loại lớp phủ:</h4>
"""
for lc_code in color_map:
    legend_html += f"""
    <p><i style="background: {color_map[lc_code]}; width: 15px; height: 15px; display: inline-block;"></i>
    {LAND_COVER_MAPPING[lc_code]}</p>
    """
legend_html += "</div>"
m.get_root().html.add_child(folium.Element(legend_html))

# Hiển thị bản đồ
m